In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

## ResNet

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
#      transforms.RandomHorizontalFlip(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./dataset/', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./dataset/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Define ResNet

In [3]:
import torch.nn as nn
import torch.nn.functional as F

net = torchvision.models.resnet50(pretrained=True)
net.fc = nn.Linear(2048, 10, bias=True)

### Define Loss Function

In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Train the Network

In [5]:
len(trainset)

50000

In [7]:
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(trainloader, 0), total=len(trainset)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')


  0%|                                                 | 0/50000 [00:00<?, ?it/s][W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.

  0%|                                      | 67/50000 [00:17<3:32:28,  3.92it/s]


KeyboardInterrupt: 

### Test Network

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 68 %


|epoch|acc|
|:----:|:----:|
|1|49%|
|2|54%|
|20|59%|

## Homework
##### (1)阅读论文，并重写ResNet-50 （https://arxiv.org/abs/1512.03385）
##### (2)阅读论文，使用VGG-16训练Ciffar-10. （https://arxiv.org/abs/1409.1556）